In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import random
import numpy as np

In [38]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic=True

In [39]:
train_transforms = transforms.Compose([

                           transforms.RandomHorizontalFlip(),

                           transforms.RandomRotation(10),

                           transforms.RandomCrop((224, 224), pad_if_needed=True),

                           transforms.ToTensor(),

                           # color channel 정규화 범위 지정

                           transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

                       ])
test_transforms = transforms.Compose([

                           transforms.CenterCrop((224, 224)),

                           transforms.ToTensor(),

                           transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

                       ])


In [41]:
train_data = datasets.ImageFolder('d:/data/catdog/train', train_transforms)
test_data = datasets.ImageFolder('d:/data/catdog/test', test_transforms)
len(train_data), len(test_data)

(8005, 2023)

In [42]:
BATCH_SIZE = 64
train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [43]:
device = torch.device('cuda')

In [44]:
import torchvision.models as models
#2015년 발표된 알고리즘
model = models.resnet18(pretrained=True).to(device)
model

c:\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [45]:
for param in model.parameters():
    param.requires_grad = False # 레이어 동결

In [46]:
print(model.fc) #input 512, output 1000

Linear(in_features=512, out_features=1000, bias=True)


In [47]:
# 이진분류이므로 out_features 2로 수정
model.fc = nn.Linear(in_features=512, out_features=2).to(device)
model.fc

Linear(in_features=512, out_features=2, bias=True)

In [48]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [21]:
arr = torch.rand(2,3,4) # 2행 3열 0~3 정수 난수
pred = arr.argmax(2, keepdim= False)
print(pred)
print(pred.shape) # 출력값의 차원이 1 감소함 (2차원=>1차원)

tensor([[1, 1, 1],
        [2, 2, 2]])
torch.Size([2, 3])


In [22]:
pred = arr.argmax(2, keepdim=True)
print(pred)
print(pred.shape) # 출력값의 차원이 감소하지 않음

tensor([[[1],
         [1],
         [1]],

        [[2],
         [2],
         [2]]])
torch.Size([2, 3, 1])


In [49]:
def calculate_accuracy(fx, y):

    preds = fx.max(1, keepdim=True)[1]

    correct = preds.eq(y.view_as(preds)).sum()

    acc = correct.float()/preds.shape[0]

    return acc

In [50]:
def train(model, device, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in iterator:

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        fx = model(x)

        loss = criterion(fx, y)

        acc = calculate_accuracy(fx, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [51]:
def evaluate(model, device, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for (x, y) in iterator:
            x = x.to(device)
            y = y.to(device)

            fx = model(x)
            loss = criterion(fx, y)
            acc = calculate_accuracy(fx, y)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [52]:
best_valid_loss = float('inf')

epochs=2

for epoch in range(epochs):
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, test_iterator, criterion)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'dog-cat.pt')

    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% |')

| Epoch: 01 | Train Loss: 0.254 | Train Acc: 90.03% | Val. Loss: 0.104 | Val. Acc: 96.42% |
| Epoch: 02 | Train Loss: 0.159 | Train Acc: 93.45% | Val. Loss: 0.086 | Val. Acc: 96.71% |


In [ ]:
torch.save(model.state_dict(), 'dog-cat.pt')

In [ ]:
model.load_state_dict(torch.load('dog-cat.pt'))
evaluate(model, device, test_iterator, criterion)